In [1]:
import os
import numpy as np
import pickle
import json 

from matplotlib import pyplot as plt 

import habitat_sim.sim
from habitat_sim.utils.common import quat_from_coeffs

from scipy.spatial.transform import Rotation as R
import magnum as mn

# Notebook to generate RGB images from Habitat Sim necessary for NeRAF

In [1]:
# close the simulator if it is already running to avoid crash
try:  
    sim.close()
except NameError:
    pass

## Set parameters

In [3]:
# Choice of scene, dataset and path
dataset = 'Replica'
scene = 'office_4'
scene_path = './sound-spaces/data/scene_datasets/replica' # update the path with yours

In [4]:
# Simulator Parameters
scene_id = scene
sim_settings = {
    "path": scene_path,
    'scene_id': scene_id,
    "scene": os.path.join(scene_path, scene_id, 'habitat', 'mesh_semantic.ply'),  # Scene path
    "scene_dataset": os.path.join(scene_path, scene_id, 'habitat', 'replica_stage.stage_config.json'),
    'navmesh': os.path.join(scene_path, scene_id, 'habitat', 'mesh_semantic.navmesh'),
    "default_agent": 0,  # Index of the default agent
    "sensor_height": 1.5,  # Height of sensors in meters, relative to the agent
    "width": 512,  # Spatial resolution of the observations
    "height": 512,
    "hfov": 90,
    "vfov": 90,
}

In [5]:
def make_backend_config(settings, sensor_specs):
    # simulator backend
    backend_cfg = habitat_sim.SimulatorConfiguration()
    backend_cfg.gpu_device_id = 0
    backend_cfg.scene_id = settings["scene"]
    backend_cfg.scene_dataset_config_file = settings["scene_dataset"]
    backend_cfg.load_semantic_mesh = True
    backend_cfg.enable_physics = False

    # agent
    agent_cfg = habitat_sim.AgentConfiguration()
    agent_cfg.sensor_specifications = sensor_specs

    return  habitat_sim.Configuration(backend_cfg, [agent_cfg])

def make_sensors(settings):
    sensor_specs = []
    # rgb sensor
    rgb_sensor_spec = habitat_sim.CameraSensorSpec()
    rgb_sensor_spec.uuid = "color_sensor"
    rgb_sensor_spec.sensor_type = habitat_sim.SensorType.COLOR
    rgb_sensor_spec.sensor_subtype = habitat_sim.SensorSubType.PINHOLE #EQUIRECTANGULAR#PINHOLE
    rgb_sensor_spec.resolution = [settings["height"], settings["width"]]
    rgb_sensor_spec.position = [0.0, settings["sensor_height"], 0.0]
    rgb_sensor_spec.orientation = [0.0, 0.0, 0.0]
    rgb_sensor_spec.hfov = mn.Deg(settings["hfov"])
    sensor_specs.append(rgb_sensor_spec)

    # registrer camera parameters
    aspect_ratio = settings["width"] / settings["height"] 
    fx = (1 / np.tan(settings["hfov"] * (np.pi/180)/ 2.))
    fy = (1 / np.tan(settings["hfov"] * (np.pi/180) / 2.)) * aspect_ratio # habitatdoc if width different height
    fx_px = settings["width"] / (2 * fx)
    fy_px = settings["height"] / (2 * fy)
    settings['near'] = rgb_sensor_spec.near
    settings['far'] = rgb_sensor_spec.far
    settings['aspect ratio'] = aspect_ratio
    settings['fx'] = fx
    settings['fy'] = fy
    settings['fx px'] =  fx_px
    settings['fy px'] = fy_px


    return sensor_specs


## Run the simulator

In [ ]:
## Create the simulator
sensor_specs = make_sensors(sim_settings)
cfg = make_backend_config(sim_settings, sensor_specs)
sim = habitat_sim.Simulator(cfg)
sim.pathfinder.load_nav_mesh(os.path.join(scene_path, scene_id, 'habitat', 'mesh_semantic.navmesh'))

In [ ]:
## Open pkl files containing eval and train poses
path2trainposes = os.path.join(scene_id, scene_id + '_Train.pkl')
path2evalposes = os.path.join(scene_id, scene_id + '_Eval.pkl')

with open(path2trainposes, 'rb') as f:
    train_poses = pickle.load(f)
with open(path2evalposes, 'rb') as f:
    eval_poses = pickle.load(f)

print('Number of train poses:', len(train_poses))
print('Number of eval poses:', len(eval_poses))

In [ ]:
## Generate the observations 

display = False 

for set_data in ['train', 'eval']:
    print('Generating:', set_data)
    if set_data == 'train':
        poses = train_poses
        train_obs = {}
    else:
        poses = eval_poses
        eval_obs = {}

    num_obs = 0
    color_sensor = sim.get_agent(0)._sensors["color_sensor"]

    for pt_idx in poses.keys():
        data = poses[pt_idx]
        angle = pt_idx[1]
        pose = data['Position']
        quat = quat_from_coeffs(data['Quaternion'])
        
        agent = sim.get_agent(0)
        new_state = sim.get_agent(0).get_state()
        new_state.position = np.array(pose) 
        new_state.rotation = quat
        new_state.sensor_states = {}
        agent.set_state(new_state, True)

        obs = sim.get_sensor_observations()
        obs['Quaternion'] = data['Quaternion'] # easier to also store them here for later use
        obs['Position'] = data['Position']
        
        if display: 
            rgb = np.array(obs["color_sensor"])
            plt.imshow(rgb)
            plt.show()
            
        num_obs += 1

        if set_data == 'eval':
            eval_obs[(tuple(pose), angle)] = obs
        else:
            train_obs[(tuple(pose), angle)] = obs
            
    print('Number of observations in', set_data, num_obs)

## Go to nerfstudio format

In [9]:
## Create transforms.json in nerfstudio format and save the images
# Load sim settings 
path2settings = os.path.join(scene_id, scene_id + '_SimParams.json')
json_data = json.load(open(path2settings, "r"))

In [10]:
# Camera intrinsics 
w = json_data["width"]
h = json_data["height"]
fl_x = json_data["fx px"]
fl_y = json_data["fy px"]
cx = w/2
cy = h/2

dict_transforms = {
    "camera_model": "OPENCV",
    "orientation_override": "none",
    "frames": []
}

In [11]:
# Directory that will contain the generated images
i = 1
imdir = os.path.join(scene_id, 'images')

#remove the directory if it exists
if os.path.exists(imdir):
    for f in os.listdir(imdir):
        os.remove(os.path.join(imdir, f))
    os.rmdir(imdir)

os.mkdir(imdir)

In [12]:
# Generate the images and the transforms.json

for set_data in ['train', 'eval']: 
    if set_data == 'train':
        obs = train_obs
    else:
        obs = eval_obs

    for k, v in obs.items():
        im_name = set_data + '_' + "frame_{:05d}.jpg".format(i)
        pos = v["Position"]
        rot = v["Quaternion"]

        matrix_rot = R.from_quat(rot).as_matrix()
        matrix_transform = np.eye(4)
        matrix_transform[:3, :3] = matrix_rot
        matrix_transform[:3, 3] = pos
        matrix_transform[3, 3] = 1

        # camera to world from right up back (camera coordinates) to left up back (world coordinates)
        matrix_transform = np.array([
            [-1, 0, 0, 0],
            [0, 0, 1, 0],
            [0, 1, 0, 0],
            [0, 0, 0, 1]
        ]).dot(matrix_transform)


        im = v["color_sensor"]
        frame = {
            "fl_x": fl_x,
            "fl_y": fl_y,
            "cx": cx,
            "cy": cy,
            "w": w,
            "h": h,
            "file_path": os.path.join(imdir, im_name),
            "transform_matrix": matrix_transform.tolist()
        }

        dict_transforms["frames"].append(frame)
        plt.imsave(os.path.join(imdir, im_name), im)
        i+=1

In [13]:
# Remove transforms.json if exists and save the new one
if os.path.exists(os.path.join(scene_id, "transforms.json")):
    os.remove(os.path.join(scene_id, "transforms.json"))
with open(os.path.join(scene_id, "transforms.json"), "w") as f:
    json.dump(dict_transforms, f, indent=2)